In [1]:
import numpy as np
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets._species_distributions import construct_grids
from sklearn.neighbors import KernelDensity

# Chargement des données
data, species_names = fetch_species_distributions(), ['Bradypus Variegatus', 'Microryzomys Minutus']

# Préparation des données
def prepare_training_data(data):
    Xtrain = np.vstack([data['train']['dd lat'], data['train']['dd long']]).T
    ytrain = np.array([d.decode('ascii').startswith('micro') for d in data['train']['species']], dtype='int')
    Xtrain *= np.pi / 180.
    return Xtrain, ytrain

def prepare_grid(data):
    xgrid, ygrid = construct_grids(data)
    X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
    land_reference = data.coverages[6][::5, ::5]
    land_mask = (land_reference > -9999).ravel()

    xy = np.vstack([Y.ravel(), X.ravel()]).T
    xy = xy[land_mask]
    xy *= np.pi / 180.

    return X, Y, land_reference, land_mask, xy

Xtrain, ytrain = prepare_training_data(data)
X, Y, land_reference, land_mask, xy = prepare_grid(data)

# Application principale Tkinter
class KDEVisualizerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Visualisation KDE sur données d'espèces")

        # Paramètres initiaux
        self.bandwidth = tk.DoubleVar(value=0.04)
        self.kernel = tk.StringVar(value="tophat")
        self.show_presence = tk.BooleanVar(value=True)

        # Interface utilisateur
        self.setup_widgets()

    def setup_widgets(self):
        # Contrôles
        controls_frame = tk.Frame(self.root)
        controls_frame.pack(side=tk.TOP, fill=tk.X)

        tk.Label(controls_frame, text="Bande passante :").pack(side=tk.LEFT)
        self.bandwidth_entry = tk.Entry(controls_frame, textvariable=self.bandwidth, width=6)
        self.bandwidth_entry.pack(side=tk.LEFT, padx=5)

        tk.Label(controls_frame, text="Noyau :").pack(side=tk.LEFT)
        self.kernel_menu = ttk.Combobox(controls_frame, textvariable=self.kernel, 
                                        values=["gaussian", "tophat", "epanechnikov"], width=12)
        self.kernel_menu.pack(side=tk.LEFT, padx=5)

        self.presence_check = tk.Checkbutton(controls_frame, text="Afficher données de présence", variable=self.show_presence)
        self.presence_check.pack(side=tk.LEFT, padx=10)

        self.plot_button = tk.Button(controls_frame, text="Tracer", command=self.plot_distributions)
        self.plot_button.pack(side=tk.LEFT, padx=10)

        # Canevas matplotlib
        self.figure = plt.Figure(figsize=(12, 6))
        self.canvas = FigureCanvasTkAgg(self.figure, master=self.root)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    def plot_distributions(self):
        self.figure.clf()

        bandwidth = self.bandwidth.get()
        kernel = self.kernel.get()

        ax1 = self.figure.add_subplot(1, 2, 1)
        ax2 = self.figure.add_subplot(1, 2, 2)

        Z_list = []
        for i in range(2):
            kde = KernelDensity(bandwidth=bandwidth, metric='haversine',
                                kernel=kernel, algorithm='ball_tree')
            kde.fit(Xtrain[ytrain == i])

            log_dens = kde.score_samples(xy)
            Z = np.full(land_mask.shape, -9999.)
            Z[land_mask] = np.exp(log_dens)
            Z = Z.reshape(X.shape)

            Z_list.append(Z)

        for idx, (ax, Z, name) in enumerate(zip([ax1, ax2], Z_list, species_names)):
            levels = np.linspace(0, Z.max(), 25)
            contour = ax.contourf(X, Y, Z, levels=levels, cmap='Reds', alpha=0.8)
            self.figure.colorbar(contour, ax=ax)

            if self.show_presence.get():
                ax.scatter(Xtrain[ytrain == idx, 1] * 180/np.pi,
                           Xtrain[ytrain == idx, 0] * 180/np.pi,
                           c='blue', s=10, alpha=0.5, label='Présences réelles')

            ax.set_title(f"Distribution : {name}")
            ax.set_xlabel("Longitude")
            ax.set_ylabel("Latitude")
            ax.legend()

        self.figure.tight_layout()
        self.canvas.draw()

# Lancement
if __name__ == "__main__":
    root = tk.Tk()
    app = KDEVisualizerApp(root)
    root.mainloop()
